In [1]:
import pandas as pd
import xgboost as xgb
from prefect import flow, task
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split

import mlflow

In [2]:
mlflow.set_tracking_uri("https://diabetes-mlflow.azurewebsites.net")
mlflow.set_experiment("diabetes-experiment")

<Experiment: artifact_location='wasbs://storagecontmflow@storageaccmflow.blob.core.windows.net/1', creation_time=1692291527723, experiment_id='1', last_update_time=1692291527723, lifecycle_stage='active', name='diabetes-experiment', tags={}>

In [3]:
data = pd.read_csv('../data/diabetes.csv')

In [4]:
data.head().transpose()

,0,1,2,3,4
Pregnancies,6.000,1.000,8.000,1.000,0.000
Glucose,148.000,85.000,183.000,89.000,137.000
BloodPressure,72.000,66.000,64.000,66.000,40.000
SkinThickness,35.000,29.000,0.000,23.000,35.000
Insulin,0.000,0.000,0.000,94.000,168.000
BMI,33.600,26.600,23.300,28.100,43.100
DiabetesPedigreeFunction,0.627,0.351,0.672,0.167,2.288
Age,50.000,31.000,32.000,21.000,33.000
Outcome,1.000,0.000,1.000,0.000,1.000


In [5]:
X = data.drop('Outcome', axis=1)  # 'Outcome' is the target column
y = data['Outcome']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [6]:
with mlflow.start_run():
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)

    # Check accuracy and other metrics
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))

    mlflow.sklearn.log_model(model, artifact_path="models")

    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification report:\n", classification_report(y_test, y_pred))

/usr/local/anaconda3/envs/diabetes-prediction/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/anaconda3/envs/diabetes-prediction/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Accuracy: 0.7532467532467533
Classification report:
               precision    recall  f1-score   support

           0       0.81      0.80      0.81        99
           1       0.65      0.67      0.66        55

    accuracy                           0.75       154
   macro avg       0.73      0.74      0.73       154
weighted avg       0.76      0.75      0.75       154

